In [1]:
# version0.0 Baseline

In [2]:
# https://archive.ics.uci.edu/ml/datasets/Census+Income
# https://archive.ics.uci.edu/ml/machine-learning-databases/adult/

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import files
uploaded = files.upload()


Saving adult.test to adult.test
Saving adult.data to adult.data


In [5]:
data = pd.read_csv('adult.data', header=None)
# data 분석
feature_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 
                 'native-country', 'income']
data.columns = feature_names

test = pd.read_csv('adult.test', skiprows=1, header=None)
test.columns = feature_names
test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [6]:
# data 분석
feature_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

train_data_origin = pd.read_csv('adult.data', header=None)
train_data_origin.columns = feature_names

test_data = pd.read_csv('adult.test', header=None, skiprows=1)
test_data.columns = feature_names

In [7]:
train_data_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [8]:
train_data_origin.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [9]:
train_data_origin.describe(include='all')

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,32561.000000,32561,3.256100e+04,32561,32561.000000,32561,32561,32561,32561,32561,32561.000000,32561.000000,32561.000000,32561,32561
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,22696,NaN,10501,NaN,14976,4140,13193,27816,21790,NaN,NaN,NaN,29170,24720
mean,38.581647,NaN,1.897784e+05,NaN,10.080679,NaN,NaN,NaN,NaN,NaN,1077.648844,87.303830,40.437456,NaN,NaN
std,13.640433,NaN,1.055500e+05,NaN,2.572720,NaN,NaN,NaN,NaN,NaN,7385.292085,402.960219,12.347429,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.178270e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.783560e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.370510e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


# EDA

In [10]:
# 생략.

In [11]:
sns.pairplot(train_data)

NameError: ignored

In [ ]:
for c in train_data.columns:
    print("{0}: {1}".format(c, len(train_data[c].unique())))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


In [ ]:
class BinaryFeatureEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.classes_ = None
        
    def fit(self, X, y=None):
        try:
            classes_ = np.unique(X)
            
            col_num = len(classes_)
            if col_num != 2:
                raise ValueError
            self.classes_ = classes_
        except:
            raise ValueError("col_num은 2여야 합니다.")
        return self
    
    def transform(self, X, y=None):
        return (X == self.classes_[1]).astype(int)
        

In [ ]:
X_train = train_data.drop('income',axis=1)
y_train = train_data['income']

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
np.unique(y_train_encoded)

In [ ]:
num_columns = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cgr_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']
bin_columns = ['sex']

num_pipeline = Pipeline([
                         ('imputer', SimpleImputer()), # 임시로 달아놓음.
                         ('scaler', StandardScaler())
])

cgr_pipeline = Pipeline([
                         ('encoder', OneHotEncoder()),
])
bin_pipeline = Pipeline([
                         ('encoder', BinaryFeatureEncoder())
])

pipeline = ColumnTransformer([
                              # ('nothing', 'passthrough', num_columns),
                              ('num_pipeline', num_pipeline, num_columns),
                              ('cgr_pipeline', cgr_pipeline, cgr_columns),
                              ('bin_pipeline', bin_pipeline, bin_columns),
])


In [32]:
X_train_ready = pipeline.fit_transform(X_train).toarray()
X_train_ready.shape

NameError: ignored

In [33]:
X_train.shape

NameError: ignored

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
!pip install xgboost
import xgboost

In [ ]:
# model selection
import tqdm
log_reg = LogisticRegression(solver='liblinear')
svm_clf = SVC(kernel='linear')
xgb = xgboost.sklearn.XGBClassifier(n_jobs=-1)
forest = RandomForestClassifier(max_depth=3, n_jobs=-1)

clf_list = [log_reg, svm_clf, xgb, forest]


In [ ]:

score_list = []
for clf in tqdm.tqdm(clf_list):
    # clf.fit(X_train_ready, y_train_encoded)
    cv_score = cross_val_score(clf, X_train_ready, y_train_encoded,cv=5, 
                               scoring='accuracy', n_jobs=-1, verbose=10).mean()
    score_list.append(
        (clf, cv_score)
    )

In [ ]:
y_train.value_counts()

In [ ]:
log_reg.fit(X_train_ready, y_train_encoded)
y_pred = log_reg.predict(X_train_ready)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy_score(y_pred, y_train_encoded)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_train_encoded, y_pred), recall_score(y_train_encoded, y_pred)

In [ ]:
f1_score(y_train_encoded, y_pred)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [16]:
from hpsklearn import HyperoptEstimator,svc
from hyperopt import fmin, tpe, space_eval, hp

In [18]:
hp.uniform?

In [31]:
# hyperopt라이브러리.
# Bayesian optimization algorithms 
# GridSearch는 우리가 임의의 파라미터를 주고 거기서, 찾는다.
# hyperort는 범위로서 파라미터 들을 전달하고, 그 범위에서 최적을 찾는다!

# https://github.com/hyperopt/hyperopt-sklearn

from hpsklearn import HyperoptEstimator,svc
from hyperopt import fmin, tpe, space_eval, hp

# 'C': hp.uniform('C', 0, 20),
#     'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
#     'gamma': hp.uniform('gamma', 0, 20),

kernels = hp.choice('my_kernel', ['linear', 'sigmoid', 'rbf'])
gamma = hp.uniform('gamma', 0.01, 100)
C = hp.uniform('C', 0.01, 100)


estim = HyperoptEstimator(
                          classifier=svc('my_svc'),
                          preprocessing=[],
                          algo=tpe.suggest
                          )

estim.fit(X_train, y_train_encoded)

NameError: ignored